In [31]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [32]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 14 14:43:31 2016

@author: tweetybaldwin
"""

# -*- coding: utf-8 -*-
"""
Created on Tue Jun 14 12:37:02 2016

@author: tweetybaldwin
"""
from optparse import OptionParser
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from netCDF4 import Dataset, MFDataset
import os

os.chdir("/vftmp/Jane.Baldwin/STEP/SYN_FIX/")
#plt.close("all")

In [33]:
#MASK
file = 'tx90pct_heatwaves_FLOR_Control_90pct_400yr_rNone_1112_yearly_summer.nc'
mask = Dataset(file, 'r')

lats = mask.variables['lat'][:]
lons = mask.variables['lon'][:]

lat_s = -90
lat_n = 90
lon_w = 0
lon_e = 360

lats = np.transpose(np.tile(lats,(lons.shape[0],1)))
weights = np.cos(np.deg2rad(lats))
lons = np.tile(lons,(lats.shape[0],1))
lats_mask = np.logical_and(lats >= lat_s, lats <= lat_n)
lons_mask = np.logical_and(lons >= lon_w, lons <= lon_e)
reg_mask = lats_mask*lons_mask

In [34]:
#SET UP PLOT

fig, ((ax1,ax2),(ax3,ax4))=plt.subplots(2,2)

ax1.set_title('Proportion Compound Days')
ax1.set_xlabel('Control [%]')
ax1.set_ylabel('2xCO2 [%]')
ax1.set_yscale('linear')
ax1.set_ylim((0,70))
ax1.set_xlim((0,40))

ax2.set_title('Proportion Compound Days')
ax2.set_xlabel('Control [%]')
ax2.set_ylabel('2xCO2 [%]')
ax2.set_yscale('linear')
ax2.set_ylim((0,100))
ax2.set_xlim((0,100))

ax3.set_title('Absolute Change')
ax3.set_ylabel('2xCO2-Control [Days]')
ax3.set_xlabel('Control')
ax3.set_yscale('linear')
ax3.set_xlim((0,60))
ax3.set_ylim((0,60))

ax4.set_title('Percent Change')
ax4.set_ylabel('2xCO2-Control [%]')
ax4.set_xlabel('Control [Days]')
ax4.set_yscale('linear')
ax4.set_xlim((0,60))
ax4.set_ylim((-100,30000))


(-100, 30000)

In [35]:
#FUNCTION FOR SYN DATA
def synscatter(mn,thres,days,color,mark):
    
    cntlfile = 't'+mn+thres+'pct_heatwaves_FLOR_Control_'+thres+'pct_400yr_rNone_'+days+'_yearly_summer.nc'
    x2file = 't'+mn+thres+'pct_heatwaves_FLOR_2XCO2_'+thres+'pct_400yr_rNone_'+days+'_yearly_summer.nc'
    synfile = 't'+mn+thres+'pct_heatwaves_FLOR_XYTave_'+thres+'pct_400yr_rNone_'+days+'_yearly_summer.nc'
    
    head = cntlfile[0:7]
    cntl = Dataset(cntlfile, 'r')
    x2 = Dataset(x2file, 'r')
    syn = Dataset(synfile,'r')
    
    ah_cntl = cntl.variables['AHWF_'+head][:]
    h1_cntl = cntl.variables['AHW1F_'+head][:]
    h2_cntl = cntl.variables['AHW2F_'+head][:]
    #h_cntl = cntl.variables['HWF_'+head][:]
    ah_2x = x2.variables['AHWF_'+head][:]
    h1_2x = x2.variables['AHW1F_'+head][:]
    h2_2x = x2.variables['AHW2F_'+head][:]
    #h_2x = x2.variables['HWF_'+head][:]
    ah_syn = syn.variables['AHWF_'+head][:]
    h1_syn = syn.variables['AHW1F_'+head][:]
    h2_syn = syn.variables['AHW2F_'+head][:]
    #h_syn = syn.variables['HWF_'+head][:]                                
    
    ah0 = np.ma.average(np.mean(ah_cntl,axis=0)[reg_mask], weights = weights[reg_mask])
    h10 = np.ma.average(np.mean(h1_cntl,axis=0)[reg_mask], weights = weights[reg_mask])
    h20 = np.ma.average(np.mean(h2_cntl,axis=0)[reg_mask], weights = weights[reg_mask])
    
    ah = np.ma.average(np.mean(ah_2x,axis=0)[reg_mask], weights=weights[reg_mask])
    h1 = np.ma.average(np.mean(h1_2x,axis=0)[reg_mask], weights=weights[reg_mask])
    h2 = np.ma.average(np.mean(h2_2x,axis=0)[reg_mask], weights = weights[reg_mask])
    
    ahs = np.ma.average(np.mean(ah_syn,axis=0)[reg_mask], weights=weights[reg_mask])
    h1s = np.ma.average(np.mean(h1_syn,axis=0)[reg_mask], weights=weights[reg_mask])
    h2s = np.ma.average(np.mean(h2_syn,axis=0)[reg_mask], weights = weights[reg_mask])
    
    cp_cntl = (h20/ah0)*100
    cp_2x = (h2/ah)*100
    cp_syn = (h2s/ahs)*100
    
    dh1 = h1-h10
    dh2 = h2-h20
    dh1p = (h1-h10)/h10*100
    dh2p = (h2-h20)/h20*100
    
    ax1.scatter(cp_cntl,cp_2x,c=color,s=100,marker=mark)
    ax1.annotate(cntlfile[49:52],(cp_cntl,cp_2x))
    
   # ax2.scatter(cp_cntl,cp_2x,c='k',s=250,marker='x')
   # ax2.scatter(cp_cntl,cp_syn,c='m',s=250,marker='x')
    
   # ax3.scatter(h1,dh1,c=color,s=250,marker='o')
   # ax3.scatter(h2,dh2,c=color,s=250,marker='*')
    
   # ax4.scatter(h1,dh1p,c=color,s=250,marker='o')
   # ax4.scatter(h2,dh2p,c=color,s=250,marker='*')

In [36]:
#T-max, 90
synscatter('x','90','1112','r','o')
synscatter('x','90','1212','r','o')
synscatter('x','90','1312','r','o')
synscatter('x','90','3136','r','o')
synscatter('x','90','3236','r','o')
synscatter('x','90','3336','r','o')
synscatter('x','90','61612','r','o')
synscatter('x','90','62612','r','o')
synscatter('x','90','63612','r','o')
synscatter('x','90','3114','r','o')
synscatter('x','90','3214','r','o')
synscatter('x','90','3314','r','o')
synscatter('x','90','6117','r','o')
synscatter('x','90','6217','r','o')

In [37]:
#T-max,95

synscatter('x','95','1112','r','*')
synscatter('x','95','1212','r','*')
synscatter('x','95','1312','r','*')
synscatter('x','95','3136','r','*')
synscatter('x','95','3236','r','*')
synscatter('x','95','3336','r','*')
synscatter('x','95','61612','r','*')
synscatter('x','95','62612','r','*')
synscatter('x','95','63612','r','*')
synscatter('x','95','3114','r','*')
synscatter('x','95','3214','r','*')
synscatter('x','95','3314','r','*')
synscatter('x','95','6117','r','*')
synscatter('x','95','6217','r','*')

In [38]:
#T-min, 90

synscatter('n','90','1112','b','o')
synscatter('n','90','1212','b','o')
synscatter('n','90','1312','b','o')
synscatter('n','90','3136','b','o')
synscatter('n','90','3236','b','o')
synscatter('n','90','3336','b','o')
synscatter('n','90','61612','b','o')
synscatter('n','90','62612','b','o')
synscatter('n','90','63612','b','o')
synscatter('n','90','3114','b','o')
synscatter('n','90','3214','b','o')
synscatter('n','90','3314','b','o')
synscatter('n','90','6117','b','o')
synscatter('n','90','6217','b','o')

In [39]:
#T-min, 95

synscatter('n','95','1112','b','*')
synscatter('n','95','1212','b','*')
synscatter('n','95','1312','b','*')
synscatter('n','95','3136','b','*')
synscatter('n','95','3236','b','*')
synscatter('n','95','3336','b','*')
synscatter('n','95','61612','b','*')
synscatter('n','95','62612','b','*')
synscatter('n','95','63612','b','*')
synscatter('n','95','3114','b','*')
synscatter('n','95','3214','b','*')
synscatter('n','95','3314','b','*')
synscatter('n','95','6117','b','*')
synscatter('n','95','6217','b','*')

In [40]:
ax1.plot([0,100],[0,100],'k')
ax2.plot([0,100],[0,100],'k')
fig = plt.gcf()
fig.set_size_inches(13,12,forward=True)
plt.show()
fig.savefig('amspres2016_scatter.pdf')